In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import pandas as pd
import numpy as np
from pathlib import Path
import tkinter as tk
import os
import random
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import tensorflow as tf
print("Dispositivos disponibles:", tf.config.list_physical_devices())

# Verificar si realmente estamos en CPU
if tf.config.list_physical_devices('GPU'):
    print("Advertencia: ¡Todavía hay una GPU activa!")

import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

import tensorflow as tf
print("Dispositivos disponibles después de forzar CPU:", tf.config.list_physical_devices())


2025-01-31 11:09:45.416411: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738318185.433718   12872 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738318185.438395   12872 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-31 11:09:45.454996: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Dispositivos disponibles: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
Dispositivos disponibles después de forzar CPU: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


2025-01-31 11:09:48.948381: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-01-31 11:09:48.948425: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:137] retrieving CUDA diagnostic information for host: pc1
2025-01-31 11:09:48.948436: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:144] hostname: pc1
2025-01-31 11:09:48.948558: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:168] libcuda reported version is: 550.120.0
2025-01-31 11:09:48.948594: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:172] kernel reported version is: 550.120.0
2025-01-31 11:09:48.948603: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:259] kernel version seems to match DSO: 550.120.0


In [2]:
# Crear una ventana oculta de Tkinter
from tkinter import filedialog

# Create a hidden Tkinter window
root = tk.Tk()
root.withdraw()  # Hide the main window

# Initial path
initial_path = Path(r'/home/felix/Escritorio/TFG/datasets_norm/goose_norm_test')


try:
    # Select the first path
    print("Select the path for 'x_train'")
    x_train_path = Path(filedialog.askdirectory(
        title="Select the folder for 'x_train'",
        initialdir=initial_path
    ))

    # Select the second path
    print("Select the path for 'y_train'")
    y_train_path = Path(filedialog.askdirectory(
        title="Select the folder for 'y_train'",
        initialdir=initial_path
    ))

    # Select the third path
    print("Select the path for 'x_val'")
    x_val_path = Path(filedialog.askdirectory(
        title="Select the folder for 'x_val'",
        initialdir=initial_path
    ))

    # Select the fourth path
    print("Select the path for 'y_val'")
    y_val_path = Path(filedialog.askdirectory(
        title="Select the folder for 'y_val'",
        initialdir=initial_path
    ))

    # Show the selected paths
    print(f"Path selected for x_train: {x_train_path}")
    print(f"Path selected for y_train: {y_train_path}")
    print(f"Path selected for x_val: {x_val_path}")
    print(f"Path selected for y_val: {y_val_path}")

except Exception as e:
    print(f"Error selecting paths: {e}")

finally:
    root.destroy()  # Ensure the Tkinter window is closed

Select the path for 'x_train'
Select the path for 'y_train'
Select the path for 'x_val'
Select the path for 'y_val'
Path selected for x_train: /home/felix/Escritorio/TFG/datasets_norm/goose_norm_test/x_train
Path selected for y_train: /home/felix/Escritorio/TFG/datasets_norm/goose_norm_test/y_train
Path selected for x_val: /home/felix/Escritorio/TFG/datasets_norm/goose_norm_test/x_val
Path selected for y_val: /home/felix/Escritorio/TFG/datasets_norm/goose_norm_test/y_val


In [3]:
# Función para cargar y procesar los datos
def load_data(x_path, y_path):
    X_data, Y_data = [], []
    num_points_list = []

    for x_file in Path(x_path).iterdir():
        if x_file.is_file() and x_file.suffix == '.csv':
            y_file = Path(y_path) / x_file.name.replace('x_', 'y_')

            x_df = pd.read_csv(x_file, header=None, skiprows=1)
            y_df = pd.read_csv(y_file, header=None, skiprows=1)

            x_features = x_df.iloc[:, [0, 1, 2]].astype(float).values
            labels = y_df.iloc[:, 0].values

            X_data.append(x_features)
            Y_data.append(labels)
            num_points_list.append(len(x_features))
    
    return X_data, Y_data, min(num_points_list)  # Retorna además el número mínimo de puntos

# Cargar datos y obtener la cantidad mínima de puntos
x_train, y_train, min_train_points = load_data(x_train_path, y_train_path)
x_val, y_val, min_val_points = load_data(x_val_path, y_val_path)

min_points = min(min_train_points, min_val_points)  # Definir la cantidad mínima global

# def downsample_data(X_data, Y_data, num_points):
#    X_downsampled, Y_downsampled = [], []
#    for x, y in zip(X_data, Y_data):
#        indices = np.random.permutation(len(x))[:num_points]  # Selección aleatoria
#       X_downsampled.append(x[indices])
#        Y_downsampled.append(y[indices])
#    return np.array(X_downsampled), np.array(Y_downsampled)

# Aplicar el downsampling
#x_train, y_train = downsample_data(x_train, y_train, min_points)

#x_val, y_val = downsample_data(x_val, y_val, min_points)

In [4]:
# Verificar las formas de los datos
print(f"x_train shape: {x_train.shape}, y_train shape: {y_train.shape}")
print(f"x_val shape: {x_val.shape}, y_val shape: {y_val.shape}")

x_train shape: (525, 119809, 3), y_train shape: (525, 119809)
x_val shape: (132, 119809, 3), y_val shape: (132, 119809)


In [7]:
# T-Net para transformación de entrada
def tnet(inputs, k):
    x = layers.Conv1D(64, kernel_size=1, activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(128, kernel_size=1, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(1024, kernel_size=1, activation='relu')(x)
    x = layers.GlobalMaxPooling1D()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dense(k * k, kernel_initializer=tf.keras.initializers.GlorotUniform())(x)
    return layers.Reshape((k, k))(x)

# Modelo PointNet mejorado
def pointnet_segmentation(num_points, num_classes):
    inputs = layers.Input(shape=(num_points, 3))

    tnet_input = tnet(inputs, 3)
    transformed_inputs = layers.Dot(axes=(2, 1))([inputs, tnet_input])

    x = layers.Conv1D(64, kernel_size=1, activation='relu')(transformed_inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Conv1D(64, kernel_size=1, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    tnet_feature = tnet(x, 64)
    transformed_features = layers.Dot(axes=(2, 1))([x, tnet_feature])

    x = layers.Conv1D(128, kernel_size=1, activation='relu')(transformed_features)
    x = layers.Conv1D(1024, kernel_size=1, activation='relu')(x)
    global_feature = layers.GlobalMaxPooling1D()(x)
    global_feature = layers.Lambda(lambda x: tf.expand_dims(x, axis=1))(global_feature)
    global_feature = layers.Lambda(lambda x: tf.tile(x, [1, num_points, 1]))(global_feature)

    x = layers.Concatenate()([transformed_features, global_feature])
    x = layers.Conv1D(512, kernel_size=1, activation='relu')(x)
    x = layers.Conv1D(256, kernel_size=1, activation='relu')(x)
    x = layers.Conv1D(128, kernel_size=1, activation='relu')(x)
    outputs = layers.Conv1D(num_classes, kernel_size=1, activation='softmax')(x)

    return models.Model(inputs=inputs, outputs=outputs)

In [8]:
num_classes = 64
model = pointnet_segmentation(min_points, num_classes)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy(),
                  tf.keras.metrics.MeanIoU(num_classes=num_classes)]
              )

In [ ]:
# Entrenamiento
epochs = 10
batch_size = 16

history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val))

2025-01-31 11:15:30.027581: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 754796700 exceeds 10% of free system memory.


Epoch 1/10


2025-01-31 11:15:30.761655: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 503197800 exceeds 10% of free system memory.
2025-01-31 11:15:36.471770: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 490737664 exceeds 10% of free system memory.
2025-01-31 11:15:36.767193: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 490737664 exceeds 10% of free system memory.
2025-01-31 11:15:36.940026: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 490737664 exceeds 10% of free system memory.
